# Naive Bayes Classifier for Classification

Help taken from machinelearningmastery

-Handle Data: Load the data from CSV file and split it into training and test datasets.

-Summarize Data: summarize the properties in the training dataset so that we can calculate probabilities and make predictions.

-Make a Prediction: Use the summaries of the dataset to generate a single prediction.

-Make Predictions: Generate predictions given a test dataset and a summarized training dataset.

-Evaluate Accuracy: Evaluate the accuracy of predictions made for a test dataset as the percentage correct out of all predictions made.

-Tie it Together: Use all of the code elements to present a complete and standalone implementation of the Naive Bayes algorithm.

Load CSV

In [215]:
import csv

def loadfile(filename):
    lines = csv.reader(open(filename,"rt", encoding='utf8'))
    dataset = list(lines)
    for i in range(1,len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset[1:]

In [216]:
filename = "diabetes.csv"
dataset = loadfile(filename)
print("loaded data file {} with {} rows".format(filename,len(dataset)))

loaded data file diabetes.csv with 768 rows


In [217]:
import random
def splitData(dataset,split_ratio):
    train_len = int(len(dataset)*split_ratio)
    train_set = []
    copy = list(dataset)
    while(len(train_set)< train_len):
        index = random.randrange(len(copy))
        train_set.append(copy.pop(index))
    return [train_set,copy]

In [218]:
dataset = [[1], [2], [3], [4], [5]]
splitRatio = 0.67
train, test = splitData(dataset, splitRatio)
print('Split {0} rows into train with {1} and test with {2}'.format(len(dataset), train, test))

Split 5 rows into train with [[3], [2], [4]] and test with [[1], [5]]


In [219]:
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

In [220]:
import math
def mean(numbers):
    numbers = list(numbers)
    for i in range(len(numbers)):
        numbers[i] = float(numbers[i])
    return sum(numbers)/float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [221]:
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    #zip(*dataset) arrangles numbers in the attribute form and then we can find summary of each attribute
    del summaries[-1]
    return summaries

In [222]:
dataset = [[1,20,0], [2,21,1], [3,22,0]]
summary = summarize(dataset)
print('Attribute summaries: {0}'.format(summary))

Attribute summaries: [(2.0, 1.0), (21.0, 1.0)]


In [223]:
def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for value in separated:
        summaries[value] = summarize(separated[value])
    return summaries

In [224]:
dataset = [[1,20,1], [2,21,0], [3,22,1], [4,22,0]]
summary = summarizeByClass(dataset)
print('Summary by class value: {0}'.format(summary))

Summary by class value: {0: [(3.0, 1.4142135623730951), (21.5, 0.7071067811865476)], 1: [(2.0, 1.4142135623730951), (21.0, 1.4142135623730951)]}


Calculate probabilities

In [225]:
import math
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [226]:
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for value in summaries:
        probabilities[value] = 1
        class_summary = summaries[value]
        for i in range(len(class_summary)):
            mean, stdev = class_summary[i]
            x = inputVector[i]
            probabilities[value] *= calculateProbability(x, mean, stdev)
    return probabilities

In [227]:
summaries = {0:[(1, 0.5)], 1:[(20, 5.0)]}
inputVector = [1.1, '?']
probabilities = calculateClassProbabilities(summaries, inputVector)
print('Probabilities for each class: {0}'.format(probabilities))

Probabilities for each class: {0: 0.7820853879509118, 1: 6.298736258150442e-05}


In [228]:
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for value in probabilities:
        probability = probabilities[value]
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = value
    return bestLabel

In [229]:
summaries = {'A':[(1, 0.5)], 'B':[(20, 5.0)]}
inputVector = [1.1, '?']
result = predict(summaries, inputVector)
print('Prediction: {0}'.format(result))

Prediction: A


In [230]:
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

In [231]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [232]:
def main():
    filename = 'diabetes.csv'
    splitRatio = 0.67
    dataset = loadfile(filename)
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingSet), len(testSet)))
    # prepare model
    summaries = summarizeByClass(trainingSet)
    # test model
    predictions = getPredictions(summaries, testSet)
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: {0}%'.format(accuracy))

main()

Split 768 rows into train=514 and test=254 rows
Accuracy: 75.98425196850394%
